In [ ]:
import numpy as np
import math
import xarray as xr
import dask
from sklearn.utils import shuffle
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import netCDF4
import pandas as pd
from sklearn.cluster import KMeans
from scipy.ndimage.filters import uniform_filter1d
bm = Basemap()

In [ ]:
Max_Scalar = np.load('/fast/gmooers/Preprocessed_Data/Centered_50_50/Space_Time_Max_Scalar.npy')
Min_Scalar = np.load('/fast/gmooers/Preprocessed_Data/Centered_50_50/Space_Time_Min_Scalar.npy')

In [ ]:
path_to_file = '/DFS-L/DATA/pritchard/gmooers/Workflow/MAPS/SPCAM/100_Days/New_SPCAM5/archive/TimestepOutput_Neuralnet_SPCAM_216/atm/hist/TimestepOutput_Neuralnet_SPCAM_216.cam.h1.2009-01-20-00000.nc'
extra_variables = xr.open_dataset(path_to_file)
lats = np.squeeze(extra_variables.LAT_20s_to_20n.values)
lons = np.squeeze(extra_variables.LON_0e_to_360e.values)

In [ ]:
amazon_lats = lats[4:14]
amazon_lons = lons[-30:-20]

In [ ]:
path_to_file = '/DFS-L/DATA/pritchard/gmooers/Workflow/MAPS/SPCAM/100_Days/New_SPCAM5/archive/TimestepOutput_Neuralnet_SPCAM_216/atm/hist/TimestepOutput_Neuralnet_SPCAM_216.cam.h1.2008-10-*'
extra_variables = xr.open_mfdataset(path_to_file)
amazon = extra_variables.CRM_W_LON_0e_to_360e_LAT_20s_to_20n[:,:,:,:,4:14,-30:-20]
amazon_lat = xr.DataArray.mean(amazon, axis = 4)
amazon_lat_lon = xr.DataArray.mean(amazon_lat, axis = 4)
oct_amazon_avg = xr.DataArray.squeeze(amazon_lat_lon)
print(oct_amazon_avg.shape)

In [ ]:
path_to_file = '/DFS-L/DATA/pritchard/gmooers/Workflow/MAPS/SPCAM/100_Days/New_SPCAM5/archive/TimestepOutput_Neuralnet_SPCAM_216/atm/hist/TimestepOutput_Neuralnet_SPCAM_216.cam.h1.2008-11-*'
extra_variables = xr.open_mfdataset(path_to_file)
amazon = extra_variables.CRM_W_LON_0e_to_360e_LAT_20s_to_20n[:,:,:,:,4:14,-30:-20]
amazon_lat = xr.DataArray.mean(amazon, axis = 4)
amazon_lat_lon = xr.DataArray.mean(amazon_lat, axis = 4)
nov_amazon_avg = xr.DataArray.squeeze(amazon_lat_lon)
print(nov_amazon_avg.shape)

In [ ]:
path_to_file = '/DFS-L/DATA/pritchard/gmooers/Workflow/MAPS/SPCAM/100_Days/New_SPCAM5/archive/TimestepOutput_Neuralnet_SPCAM_216/atm/hist/TimestepOutput_Neuralnet_SPCAM_216.cam.h1.2008-12-*'
extra_variables = xr.open_mfdataset(path_to_file)
amazon = extra_variables.CRM_W_LON_0e_to_360e_LAT_20s_to_20n[:,:,:,:,4:14,-30:-20]
amazon_lat = xr.DataArray.mean(amazon, axis = 4)
amazon_lat_lon = xr.DataArray.mean(amazon_lat, axis = 4)
dec_amazon_avg = xr.DataArray.squeeze(amazon_lat_lon)
print(dec_amazon_avg.shape)

In [ ]:
path_to_file = '/DFS-L/DATA/pritchard/gmooers/Workflow/MAPS/SPCAM/100_Days/New_SPCAM5/archive/TimestepOutput_Neuralnet_SPCAM_216/atm/hist/TimestepOutput_Neuralnet_SPCAM_216.cam.h1.2009-01-*'
extra_variables = xr.open_mfdataset(path_to_file)
amazon = extra_variables.CRM_W_LON_0e_to_360e_LAT_20s_to_20n[:,:,:,:,4:14,-30:-20]
amazon_lat = xr.DataArray.mean(amazon, axis = 4)
amazon_lat_lon = xr.DataArray.mean(amazon_lat, axis = 4)
jan_amazon_avg = xr.DataArray.squeeze(amazon_lat_lon)
print(jan_amazon_avg.shape)

In [11]:
amazon_avg_fall = np.concatenate((oct_amazon_avg, nov_amazon_avg), axis = 0)
print("fall")
amazon_avg_winter = np.concatenate((dec_amazon_avg, jan_amazon_avg), axis = 0)
print("winter")
amazon_avg = np.concatenate((amazon_avg_fall, amazon_avg_winter), axis = 0)
print("all")
print(amazon_avg.shape)

KeyboardInterrupt: 

In [ ]:
np.save('/fast/gmooers/Preprocessed_Data/Global_Patterns/Raw_Amazon.npy', amazon_avg)

In [18]:
day_composite = np.zeros(shape=(96,30,128))
day_composite[:,:,:] = np.nan

In [19]:
for i in range(96):
    print(i)
    day_composite[i,:,:] = np.nanmean(amazon_avg[i::96,:],axis=0)

0
1
2


KeyboardInterrupt: 

In [ ]:
def image_reconstruct(image, N):
    #plot_generator(image)
    image = np.squeeze(image)
    chunk = image[17:22,:]
    mean_chunk = np.sum(np.abs(chunk), axis =0)
    means = uniform_filter1d(mean_chunk, size=N)
    ind = np.argmax(means)
    middle = 63
    if ind <63:
        #print("A")
        splice_ind = middle - ind
        img_a = image[:,-splice_ind:]
        img_b = image[:,:-splice_ind]
        image = np.concatenate((img_a,img_b), axis=1)
    if ind>64:
        #print("B")
        splice_ind = ind-middle
        img_a = image[:,:splice_ind]
        img_b = image[:,splice_ind:]
        image = np.concatenate((img_b,img_a), axis=1)
    #plot_generator(image)
    
    return image

def new_datasets(images, N):
    new_dataset = np.zeros(shape=(len(images), len(images[0]),len(images[0][0])))
    new_dataset[:,:,:]=np.nan
    for i in range(len(images)):
        new_image = image_reconstruct(images[i,:,:],N)
        new_dataset[i,:,:]=new_image
    
    return new_dataset

In [ ]:
centered_timestep = new_datasets(day_composite, 10)

In [ ]:
scaled_day = np.interp(centered_timestep, (Min_Scalar, Max_Scalar), (0, +1))

In [ ]:
np.save('/fast/gmooers/Preprocessed_Data/Global_Patterns/Scaled_Amazon_Diurnal_Composite.npy', scaled_day)